In [8]:
import pandas as pd
import os
from binance.client import Client
import datetime, time

# howLong = 2 * 365
#     # Calculate the timestamps for the binance api function
# untilThisDate = datetime.datetime.now()
# sinceThisDate = untilThisDate - datetime.timedelta(days = howLong)
# client = Client(api_key, api_secret)
#     # Execute the query from binance - timestamps must be converted to strings !
# candle = client.get_historical_klines("BNBBTC", Client.KLINE_INTERVAL_1MINUTE, str(sinceThisDate), str(untilThisDate))

#     # Create a dataframe to label all the columns returned by binance so we work with them later.
# df = pd.DataFrame(candle, columns=['dateTime', 'open', 'high', 'low', 'close', 'volume', 'closeTime', 'quoteAssetVolume', 'numberOfTrades', 'takerBuyBaseVol', 'takerBuyQuoteVol', 'ignore'])
#     # as timestamp is returned in ms, let us convert this back to proper timestamps.
# #self.df.dateTime = pd.to_datetime(self.df.dateTime, unit='ms').dt.strftime(Constants.DateTimeFormat)
# df = df.set_index('dateTime', inplace=True)

#     # Get rid of columns we do not need
# df = df.drop(['closeTime', 'quoteAssetVolume', 'numberOfTrades', 'takerBuyBaseVol','takerBuyQuoteVol', 'ignore'], axis=1)

df = pd.read_csv("LTCUSDT-5m-2021-11.csv")
df.columns = ['open_time', 'open', 'high', 'low', 'close', 'volume', 'close_time', 'quote_volume', 'count', 'taker_buy_volume', 'taker_buy_quote_volume', 'ignore']
df = df.set_index('open_time')
df = df.drop(['close_time', 'ignore'], axis=1)

,open,high,low,close,volume,quote_volume,count,taker_buy_volume,taker_buy_quote_volume
open_time,,,,,,,,,
1635725100000,191.43,192.02,191.37,192.02,2793.815,5.355669e+05,1297,1385.168,2.655765e+05
1635725400000,192.02,193.32,191.93,193.01,10168.770,1.960809e+06,3851,5928.260,1.143253e+06
1635725700000,192.98,193.76,192.92,193.62,8579.165,1.659771e+06,2904,3799.094,7.349429e+05
1635726000000,193.60,193.72,193.02,193.37,4839.350,9.358926e+05,1951,2065.522,3.995075e+05
1635726300000,193.36,193.69,193.31,193.39,4937.114,9.553264e+05,1758,2668.693,5.164217e+05
...,...,...,...,...,...,...,...,...,...
1638315300000,208.33,208.91,208.15,208.77,4036.863,8.414236e+05,1373,1840.337,3.837659e+05
1638315600000,208.78,209.18,208.77,209.00,2471.871,5.165802e+05,1203,1103.075,2.305155e+05
1638315900000,209.00,209.19,207.60,207.73,5946.212,1.237775e+06,2159,1900.718,3.957291e+05


### Calculating MACD

In [11]:
# Get the 26-day EMA of the closing price
k = df['close'].ewm(span=12, adjust=False, min_periods=12).mean()
# Get the 12-day EMA of the closing price
d = df['close'].ewm(span=26, adjust=False, min_periods=26).mean()
# Subtract the 26-day EMA from the 12-Day EMA to get the MACD
macd = k - d
# Get the 9-Day EMA of the MACD for the Trigger line
macd_s = macd.ewm(span=9, adjust=False, min_periods=9).mean()
# Calculate the difference between the MACD - Trigger for the Convergence/Divergence value
macd_h = macd - macd_s
# Add all of our new values for the MACD to the dataframe
df['macd'] = df.index.map(macd)
df['macd_h'] = df.index.map(macd_h)
df['macd_s'] = df.index.map(macd_s)

,open,high,low,close,volume,quote_volume,count,taker_buy_volume,taker_buy_quote_volume,macd,macd_h,macd_s
open_time,,,,,,,,,,,,
1635725100000,191.43,192.02,191.37,192.02,2793.815,5.355669e+05,1297,1385.168,2.655765e+05,NaN,NaN,NaN
1635725400000,192.02,193.32,191.93,193.01,10168.770,1.960809e+06,3851,5928.260,1.143253e+06,NaN,NaN,NaN
1635725700000,192.98,193.76,192.92,193.62,8579.165,1.659771e+06,2904,3799.094,7.349429e+05,NaN,NaN,NaN
1635726000000,193.60,193.72,193.02,193.37,4839.350,9.358926e+05,1951,2065.522,3.995075e+05,NaN,NaN,NaN
1635726300000,193.36,193.69,193.31,193.39,4937.114,9.553264e+05,1758,2668.693,5.164217e+05,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
1638315300000,208.33,208.91,208.15,208.77,4036.863,8.414236e+05,1373,1840.337,3.837659e+05,-0.286401,-0.017246,-0.269155
1638315600000,208.78,209.18,208.77,209.00,2471.871,5.165802e+05,1203,1103.075,2.305155e+05,-0.257096,0.009648,-0.266743
1638315900000,209.00,209.19,207.60,207.73,5946.212,1.237775e+06,2159,1900.718,3.957291e+05,-0.332516,-0.052618,-0.279898


### Weak learner